In [1]:
%cd '/content/drive/My Drive/ML/CS114.K21.KHTN/Final/'

/content/drive/My Drive/ML/CS114.K21.KHTN/Final


In [2]:
from dataloader.simpledatasetloader import SimpleDatasetLoader
from preprocessing.simplepreprocessor import SimplePreprocessor

import numpy as np 
import h5py
import os
from imutils import paths

from keras.models import load_model

Using TensorFlow backend.


In [3]:
path_image = ['Test/Mango.jpg']

In [4]:
print(path_image)

['Test/Mango.jpg']


In [5]:
model_lenet = 'H5PY/model/lenet_weights.hdf5'

In [ ]:
model_lenet = load_model(model_lenet)

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model_lenet.predict(data)
print(predictions.argmax(axis=1)[0])
               

[INFO] evaluating network...
8


In [ ]:
# initialize image preprocessor
sp = SimplePreprocessor(32,32)
sdl = SimpleDatasetLoader(preprocessors=[sp])

# load dataset from memory
(data, labels) = sdl.load(path_image,verbose=1)

In [ ]:
data.shape

(1, 32, 32, 3)

In [ ]:

from flask import Flask, render_template, request
from io import BytesIO
from keras.preprocessing import image
from keras.preprocessing.image import array_to_img, img_to_array
from keras.models import load_model
import os
from PIL import Image
import numpy as np
from base64 import b64encode

from flask_bootstrap import Bootstrap
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileRequired, FileAllowed
from wtforms import SubmitField

# code which helps initialize our server
app = Flask(__name__,template_folder=r'/content/drive/My Drive/ML/CS114.K21.KHTN/Final/Flask')
app.config['SECRET_KEY'] = 'any secret key'

bootstrap = Bootstrap(app)

saved_model = load_model("H5PY/model/lenet_weights.hdf5")
saved_model._make_predict_function()

class UploadForm(FlaskForm):
    photo = FileField('Upload an image',validators=[FileAllowed(['jpg', 'png', 'jpeg'], u'Image only!'), FileRequired(u'File was empty!')])
    submit = SubmitField(u'Predict')

def preprocess(img):
    width, height = img.shape[0], img.shape[1]
    img = image.array_to_img(img, scale=False)

    desired_width, desired_height = 32, 32

    if width < desired_width:
   	 desired_width = width
    start_x = np.maximum(0, int((width-desired_width)/2))

    img = img.crop((start_x, np.maximum(0, height-desired_height), start_x+desired_width, height))
    img = img.resize((32, 32))

    img = image.img_to_array(img)
    return img

@app.route('/', methods=['GET','POST'])
def predict():
    form = UploadForm()
    if form.validate_on_submit():
   	 print(form.photo.data)
   	 image_stream = form.photo.data.stream
   	 original_img = Image.open(image_stream)
   	 img = image.img_to_array(original_img)
   	 img = preprocess(img)
   	 img = np.expand_dims(img, axis=0)
   	 prediction = saved_model.predict_classes(img)

   	 if (prediction[0][0]==0):
   		 result = 0
   	 else:
   		 result = 1

   	 byteIO = BytesIO()
   	 original_img.save(byteIO, format=original_img.format)
   	 byteArr = byteIO.getvalue()
   	 encoded = b64encode(byteArr)

   	 return render_template('result.html', result=result, encoded_photo=encoded.decode('ascii'))

    return render_template('index.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!pip3 install flask_wtf

     |████████████████████████████████| 174kB 4.9MB/s 


In [ ]:
!pip3 install flask_bootstrap

     |████████████████████████████████| 460kB 3.2MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-cp36-none-any.whl size=460124 sha256=c079bed0d74b9c4d89eb4c2a6babcd4b82c9b7b337ecb24da1368121b764f86f
  Stored in directory: /root/.cache/pip/wheels/e8/b9/93/ef6ac3b8ead2d72cbcc042b9d58b613aaf47e533b9dc5b5999
  Created wheel for visitor: filename=visitor-0.1.3-cp36-none-any.whl size=3932 sha256=bfeecd5fb514cb8db874d7093f0e187aa447c2facfae82f8aa5608e64d808522
  Stored in directory: /root/.cache/pip/wheels/68/b0/a2/cc8c7cf94ca3d1088a7d2e27936c1e0da170e05f560973e8dd
Successfully built flask-bootstrap visitor


In [ ]:
import os
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

In [ ]:
app = Flask(__name__,template_folder=r'/content/drive/My Drive/ML/CS114.K21.KHTN/Final/Flask')

In [ ]:
from dataloader.simpledatasetloader import SimpleDatasetLoader
from preprocessing.simplepreprocessor import SimplePreprocessor
from Feature_Extraction.hog import HOG
from Feature_Extraction.localbinarypatterns import LocalBinaryPatterns
import joblib
import numpy as np 
import h5py
import os
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from flask_ngrok import run_with_ngrok
from skimage.transform import resize

In [ ]:
!pip install flask_ngrok

In [ ]:
model_path = 'H5PY/model/svm_normal.sav'

In [ ]:
model = joblib.load(model_path)

In [ ]:
# initialize image preprocessor
sp = SimplePreprocessor(32,32)
sdl = SimpleDatasetLoader(preprocessors=[sp])

In [ ]:
app = Flask(__name__, template_folder=r'/content/drive/My Drive/ML/CS114.K21.KHTN/Final/Flask')
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/', methods=['GET', 'POST'])
def main_page():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join('uploads', filename))
        return redirect(url_for('prediction', filename=filename))
    return render_template('index.html')

@app.route('/prediction/<filename>')
def prediction(filename):
    
    #Step 1
    my_image = plt.imread(os.path.join('uploads', filename))
    #Step 2
    my_image_re = resize(my_image, (1,3072))
    predictions = model.predict(my_image_re)
    
    return render_template('predict.html', predictions=predictions)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://11e9f93dfd39.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Jul/2020 19:01:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 19:01:37] "GET /favicon.ico HTTP/1.1" 404 -
[2020-07-30 19:01:56,542] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint]

In [ ]:
app = Flask(__name__,template_folder=r'/content/drive/My Drive/ML/CS114.K21.KHTN/Final/Flask')
run_with_ngrok(app)

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def main_page():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join('uploads', filename))
        return redirect(url_for('prediction', filename=filename))
    return render_template('index.html')

In [ ]:
@app.route('/prediction/<filename>')
def prediction(filename):
    
    #Step 1
    my_image = plt.imread(os.path.join('uploads', filename))
    #Step 2
    my_image_re = resize(my_image, (1,3072))
    predictions = model.predict(my_image_re)
    
    return render_template('predict.html', predictions=predictions)

In [ ]:
if __name__ == '__main__':
    app.run()